## Prompt Engineering

In [232]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
import os
import re

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [233]:
llm = ChatOpenAI(temperature=0.3,
                 openai_api_key=OPENAI_API_KEY,
                 model_name='gpt-3.5-turbo-0613',
                )

In [264]:
def generate_response(llm: ChatOpenAI, quant_topic: str, quant_title: str) -> tuple[str,str]:
    """Generate AI Twitter Content for QuantPy Media Channels

    Parameters:
        - llm:  pre-trained ChatOpenAi large language model
        - quant_topic: Topic in Quant Finance
        - quant_topic: Topic in Quant Finance

    Returns:
        - tuple[long response,short reposonse]: Chat GPT long and short responses
    """
    # System Template for LLM to follow
    system_template = """
        You are an incredibly wise and smart quantitative analyst that lives and breathes the world of quantitative finance.
        Your goal is to writing short-form content for twitter given a `topic` in the area of quantitative finance and a `title` from the user.
        
        % RESPONSE TONE:

        - Your response should be given in an active voice and be opinionated
        - Your tone should be serious w/ a hint of wit and sarcasm
        
        % RESPONSE FORMAT:
        
        - Be extremely clear and concise
        - Respond in short phrases
        - No longer than 30 words total for entire reponse template
        - Make phrases no longer than 7 words in total.
        - No longer than total of 280 characters (counting spaces and other characters)
        - Do not respond with emojis
        
        % RESPONSE CONTENT:

        - Include specific examples of where this is used in the quantitative finance space
        - If you don't have an answer, say, "Sorry, I'll have to ask the Quant Finance Gods!"    

        % RESPONSE TEMPLATE:

        - Here’s a condensed structure tailored for Twitter's 280-character limit: 
            Hook: Captivate with a one-liner.
            Intro: Briefly introduce the topic.
            Explanation: Simplify the core idea.
            Application: Note real-world relevance.
            Closing: Reflective one-liner.
            Action: Short engagement call.
            Engagement: Quick question.
    
    """
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    # human template for input
    human_template="topic to write about is {topic}, and the title will be {title}. Keep the total response under 30 words total! respond in short phrases, only one sentence maximium per line no more than 6 words"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template, 
                                                                    input_variables=["topic", "title"])

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, 
                                                    human_message_prompt])

    # get a chat completion from the formatted messages
    final_prompt = chat_prompt.format_prompt(topic=quant_topic, 
                                             title=quant_title).to_messages()
    first_response = llm(final_prompt).content

    ai_message_prompt = AIMessagePromptTemplate.from_template(first_response)

    # reminder of length
    reminder_template="This was good, but way too long, please make your response much more concise and much shorter! Make phrases no longer than 5 words in total. Please maintain the existing template."
    reminder_prompt = HumanMessagePromptTemplate.from_template(reminder_template)

    chat_prompt2 = ChatPromptTemplate.from_messages([system_message_prompt, 
                                                     human_template, 
                                                     ai_message_prompt, 
                                                     reminder_prompt])

    # get a chat completion from the formatted messages
    final_prompt = chat_prompt2.format_prompt(topic=quant_topic, 
                                              title=quant_title).to_messages()
    short_response = llm(final_prompt).content

    return first_response, short_response

In [265]:
first_response, short_response = generate_response(llm, quant_topic='Time Value of Money', quant_title='Unveiling the Magic of Compounding: Time Value of Money')

In [266]:
count_length = lambda d: sum(len(d[val]) for val in d)
count_words = lambda d: sum(len(re.findall(r'\w+', d[val])) for val in d)
key_list=["Hook", "Intro", "Explanation", "Application", "Closing", "Action", "Engagement"]
def extract_tweet(openai_tweet: str, key_list: list) -> dict:
    """Creates dictionary from Openai response using keyword template

    Parameters:
        - openai_tweet: 
        - key_list: list key words used for searching reponse template

    Returns:
        - dictionary: templated tweet
    """
    template = {}
    # Iterate through key list
    for i, key in enumerate(key_list):
        # find starting position
        start = openai_tweet.find(key_list[i])+len(key_list[i])+2
        if i != len(key_list) - 1:
            # using ending position, subset str and append to template
            end = openai_tweet.find(key_list[i+1])
            line = openai_tweet[start:end]
            template[key_list[i]] = line
        else:
            # if final word in list, only subsection by start word
            template[key_list[i]] = openai_tweet[start:]
    return template

In [270]:
extract_tweet(first_response, key_list)

{'Hook': 'Discover the enchanting power of compounding!\n',
 'Intro': 'Unveiling the Magic of Compounding: Time Value of Money.\n',
 'Explanation': 'Money today is worth more than money tomorrow due to the potential to earn interest or returns.\n',
 'Application': 'Used in investing, loans, and financial planning to determine the present and future value of cash flows.\n',
 'Closing': 'Watch your money grow and multiply!\n',
 'Action': "Start investing and harness compounding's power.\n",
 'Engagement': 'How do you make the most of compounding?'}

In [269]:
extract_tweet(short_response, key_list)

{'Hook': 'Unveil the magic of compounding!\n',
 'Intro': 'Time Value of Money.\n',
 'Explanation': 'Money today is worth more.\n',
 'Application': 'Investing, loans, financial planning.\n',
 'Closing': 'Watch your money multiply!\n',
 'Action': "Start harnessing compounding's power.\n",
 'Engagement': 'How to maximize compounding?'}

## Preprocessing content-ideas

Take text file in table structure from GPT and process it by line divisors.
Then create dicitonary and store in desired format back to text file.

In [271]:
file = open("content-ideas.txt", "r")
quant_tweets = {}

for line_no, line in enumerate(file.readlines()):
    # start 2nd row to avoid heading and underlines
    if line_no > 1:
        # split on line divisors
        items = line.split('|')
        # capture and ensure int and str formatting
        tweet_no = int(items[1])
        quant_topic = items[2].strip()
        quant_title = items[3].strip().strip('"')
        # store within dict
        quant_tweets[tweet_no] = {}
        quant_tweets[tweet_no]['topic'] = quant_topic
        quant_tweets[tweet_no]['title'] = quant_title
        # print tweet no, topic and title
        print(f"{tweet_no}_{quant_topic}_{quant_title}")
file.close()

# storing in desired format
with open('quants_tweets.txt', 'w') as f:
    for tweet_no, tweet_info in quant_tweets.items():
        tweet_repr = str(tweet_no)+'|'+tweet_info['topic']+'|'+tweet_info['title']+'|FALSE|FALSE|FALSE|\n'
        f.write(tweet_repr)

1_Time Value of Money_Unveiling the Magic of Compounding: Time Value of Money
2_Risk and Return_Playing the Odds: Understanding Risk and Return
3_Modern Portfolio Theory_Crafting the Perfect Portfolio: An Intro to MPT
4_Black-Scholes Model_The Black-Scholes Legacy: Revolutionizing Option Pricing
5_Multifactor Models_Beyond Beta: Exploring Multifactor Models
6_Copula Models_Bridging Dependencies: An Insight into Copula Models
7_Stochastic Calculus_The Dance of Chance: Delving into Stochastic Calculus
8_Ito's Lemma_Ito's Insight: Unpacking Ito's Lemma
9_Quantitative Risk Management_Taming Uncertainty: Quantitative Risk Management Essentials
10_Simple Moving Average Strategy_Smooth Moves: SMA Trading Strategy Explained
11_Backtesting_Rewinding The Market: The Art of Backtesting
12_Transaction Costs and Slippage_The Hidden Costs: Navigating Transaction Costs and Slippage
13_Portfolio Optimization_Balancing Act: Portfolio Optimization Techniques
14_QuantLib Introduction_Toolkit Talk: A Dive

In [354]:
import time
from enum import Flag
from copy import deepcopy
from json import dumps, loads
from collections import deque
from dataclasses import dataclass, asdict, field

class Boolean(Flag):
    TRUE = True
    FALSE = False

@dataclass
class Tweet:
    Hook: str
    Intro: str
    Explanation: str
    Application: str
    Closing: str
    Action: str
    Engagement: str

    @classmethod
    def from_dict(cls, tweet_d: dict):
        # return class
        return cls(
            Hook=tweet_d['Hook'],
            Intro=tweet_d['Intro'],
            Explanation=tweet_d['Explanation'],
            Application=tweet_d['Application'],
            Closing=tweet_d['Closing'],
            Action=tweet_d['Action'],
            Engagement=tweet_d['Engagement']
        )

@dataclass
class TrackTweet:
    """Class for keeping track of Tweets"""
    id: int
    topic: str
    title: str
    sent_status: Boolean = Boolean.FALSE
    gen_status: Boolean = Boolean.FALSE
    tweet: Tweet = field(init=False, repr=False)

    def __lt__(self, other):
        return (
            self.sent_status.value, self.id
        ) < (
            other.sent_status.value, other.id
        )

    @classmethod
    def from_str(cls, tweet_line: str):
        # underscores used to indicate unpacked variables, only used internally
        _id, _topic, _title, _sent_status, _gen_status, _tweet, _next_line = tweet_line.split('|')
        # convert status TRUE/FALSE to Enum Representation
        _sent_status_bool=Boolean.TRUE if _sent_status == Boolean.TRUE.name else Boolean.FALSE
        # confirm if tweet already written or not, if so load previously written tweet
        _gen_status_bool=Boolean.TRUE if _gen_status == Boolean.TRUE.name else Boolean.FALSE
        # init class without tweet
        _trackTweet = cls(
                id=int(_id),
                topic=_topic,
                title=_title,
                sent_status=_sent_status_bool,
                gen_status=_gen_status_bool
            )
        
        if _gen_status_bool:
            # return class with written tweet
            _trackTweet.tweet=Tweet.from_dict(loads(_tweet))

        return _trackTweet

    def to_str(self):
        _part_1 = f"{self.id}|{self.topic}|{self.title}|{self.sent_status.name}|{self.gen_status.name}|"
        _part_2 = f"{dumps(asdict(self.tweet)) if hasattr(self, 'tweet') else 'FALSE'}|\n"
        return _part_1 + _part_2

    def update_status(self, new_status: Boolean):
        self.sent_status = new_status
    
@dataclass
class TweetQueue:
    tweets: list[TrackTweet] = field(default_factory=list)

    def __len__(self):
        return len(self.tweets)

    def __iter__(self):
        yield from self.tweets

    @property
    def tweets_not_sent(self):
        return [tweet for tweet in self.tweets if not tweet.sent_status]

    @property
    def tweets_not_generated(self):
        return [tweet for tweet in self.tweets if not tweet.gen_status]

    def enqueue(self, tweet):
        # print(f"{tweet.to_str()} will be added.")
        self.tweets.append(tweet)

    def dequeue(self):
        # print(f"{self.tweets[0].to_str()} will be removed.")
        return self.tweets.popleft()
        
    @classmethod
    def from_text_file(cls, text_file):
        _tweets = cls()
        for tweet_line in open(text_file, 'r'):
            tweet = TrackTweet.from_str(tweet_line)
            _tweets.enqueue(tweet)
        return _tweets

    def to_text_file(self, text_file):
        with open(text_file, 'w') as f:
            for tweet in self.tweets:
                tweet_line = tweet.to_str()
                f.write(tweet_line)
        

In [360]:
# set up logger

import logging

# Get logger
logger = logging.getLogger("my logger")

# Create a handler
c_handler = logging.StreamHandler()

# link handler to logger
logger.addHandler(c_handler)

# Set logging level to the logger
logger.setLevel(logging.DEBUG) # <-- THIS!

# test
logger.debug('This is a debug message') # WILL WORK

This is a debug message


In [367]:
import logging

# Get logger
logger = logging.getLogger("twitter-bot")
# define a Handler which writes INFO messages or higher to the sys.stderr
console = logging.StreamHandler()
# add the handler to the root logger
logger.addHandler(console)
# set up logging to file
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%y-%m-%d %H:%M',
                    filename='twitter-bot.log',
                    filemode='w')

logger.setLevel(logging.DEBUG)


# logging.basicConfig(filename='app.log', filemode='w', format='%(asctime)s - %(levelname)s - %(message)s', level=logging.DEBUG)
logger.info('This will get logged to a file')

## First step is to import file of topics and ides into TweetQueue
tweetQueue = TweetQueue.from_text_file('quants_tweets.txt')
# tweetQueue

This will get logged to a file
This will get logged to a file
This will get logged to a file
This will get logged to a file
This will get logged to a file
This will get logged to a file
This will get logged to a file
This will get logged to a file
This will get logged to a file


In [359]:
for quant_tweet_idea in tweetQueue.tweets_not_sent:
    logging.info(5*'*'+'Tweet to Gen'+5*'*')
    logging.info(quant_tweet_idea)
    first_response, short_response = generate_response(llm, quant_topic=quant_tweet_idea.topic, quant_title=quant_tweet_idea.title)
    logging.info(5*'*'+'First Draft'+5*'*')
    first_draft = extract_tweet(first_response, key_list)
    logging.info(f"Content Length: {count_length(first_draft)} Tweet Dict {first_draft}")
    logging.info(5*'*'+'Second Draft'+5*'*')
    short_response = extract_tweet(short_response, key_list)
    logging.info(f"Content Length: {count_length(short_response)} Tweet Dict {short_response}")

    if count_length(first_draft) < 280:
        quant_tweet_idea.tweet = Tweet.from_dict(first_draft)
        quant_tweet_idea.gen_status = Boolean.TRUE

    elif count_length(short_response) < 280:
        quant_tweet_idea.tweet = Tweet.from_dict(short_response)
        quant_tweet_idea.gen_status = Boolean.TRUE

    else:
        logging.info('Value issue with Tweet')
        
    logging.info(50*'-')
    time.sleep(20)

KeyboardInterrupt: 

In [ ]:
tweetQueue.to_text_file('quants_tweets_check.txt')

In [355]:
tweetQueue.to_text_file('quants_tweets_check.txt')

In [356]:
tweetQueue.tweets[0]

TrackTweet(id=1, topic='Time Value of Money', title='Unveiling the Magic of Compounding: Time Value of Money', sent_status=<Boolean.FALSE: False>, gen_status=<Boolean.FALSE: False>)

In [357]:
tweetQueue2= TweetQueue.from_text_file('quants_tweets_check.txt')
tweetQueue2.tweets[0]

TrackTweet(id=1, topic='Time Value of Money', title='Unveiling the Magic of Compounding: Time Value of Money', sent_status=<Boolean.FALSE: False>, gen_status=<Boolean.FALSE: False>)

In [24]:
tweetQueue == tweetQueueVerify
print(len(tweetQueue))
tweetQueue.tweets[0]
tweetQueue.tweets[0].update_status(Boolean.TRUE)
print(len(tweetQueue))

NameError: name 'tweetQueueVerify' is not defined

In [25]:
quant_tweet_idea = tweetQueue.tweets_not_sent[0]
first_response, short_response = generate_response(llm, quant_topic=quant_tweet_idea.topic, quant_title=quant_tweet_idea.title)
first_draft = extract_tweet(first_response)
short_response = extract_tweet(short_response)

IndexError: list index out of range

In [344]:
list_not_sent = TweetQueue.list_not_sent
list_not_sent

In [329]:
quant_tweet_idea = TweetQueue.list_not_sent:
    first_response, short_response = generate_response(llm, quant_topic=quant_tweet_idea.topic, quant_title=quant_tweet_idea.title)
    first_draft = extract_tweet(first_response)
    short_response = extract_tweet(short_response)

SyntaxError: invalid syntax (2576123844.py, line 1)